In [1]:
from cameo import load_model
import cobra, pandas as pd


In [2]:
#carregando o modelo
model = load_model('iYL1228_1_modificado.xml')

In [3]:
def ajustar_meio(fonte_carbono, qtd_carbono, qtd_oxigenio, objetivo):
    global model    
    
    fonte_dict = {
        # glicose
        "glicose": "EX_glc__D_e","glicose": "EX_glc__D_e","gli": "EX_glc__D_e", "glucose": "EX_glc__D_e", "glicose": "EX_glc__D_e", "glicose": "EX_glc__D_e",
        # glicerol
        "glicerol": "EX_glyc_e","gly": "EX_glyc_e", "glycerol": "EX_glyc_e", "glicerol": "EX_glyc_e", "glicerol": "EX_glyc_e"
    }

    busca = fonte_carbono.lower() #transforma em letras minúsculas

    if busca in fonte_dict:
        fonte_id = fonte_dict[busca]
    else:
        try:
            fonte_id = model.reactions.get_by_id(fonte_carbono).id
        except KeyError:
            raise ValueError(f"Fonte de carbono '{fonte_carbono}' não encontrada no modelo.")


    c = model.reactions.get_by_id(fonte_id)
    c.lower_bound = -qtd_carbono
    c.upper_bound = qtd_carbono + 0.01


    # Ajuste do oxigênio
    if "EX_o2_e" in model.reactions:
        reac_o2 = model.reactions.get_by_id("EX_o2_e")
        reac_o2.lower_bound = -qtd_oxigenio
        reac_o2.upper_bound = qtd_oxigenio + 0.01
    else:
        raise ValueError("Reação 'EX_o2_e' não encontrada.")

    # Definir objetivo
    if objetivo in model.reactions:
        model.objective = objetivo
    else:
        raise ValueError(f"Reação '{objetivo}' não encontrada para função objetivo.")


In [4]:
import pandas as pd

# Lista de reações a monitorar
list_monitoramento = ["GAPD", "MDH", "13PPDH", "DCTR", "BTDD_RS", "BTDD_SS", "ALCD2x", "LDH_D", "ALCD19", "GDH_R"]

# DataFrames globais
df_monitoramento_total = pd.DataFrame()
df_top20_total = pd.DataFrame()

def monitorar_fluxos(sol, frac, biomassa_min, alvo):
    """
    Guarda fluxos monitorados e top 20, com cabeçalho identificando a simulação.
    """
    global df_monitoramento_total, df_top20_total

    # Biomassa e alvo atuais
    biomassa_val = sol.fluxes.get("BIOMASS_", None)
    alvo_val = sol.fluxes.get(alvo, None)

    # ----- Cabeçalho da simulação -----
    header_text = f"Simulação: {frac} | Biomassa obtida: {biomassa_val:.4f} | Alvo ({alvo}): {alvo_val:.4f}"
    
    # ----- Fluxos monitorados -----
    dados_monitoramento = {
        "Reação": list_monitoramento,
        "Fluxo (mmol/gDW/h)": [sol.fluxes.get(rxn_id, None) for rxn_id in list_monitoramento]
    }
    df_header = pd.DataFrame({"Reação": [header_text], "Fluxo (mmol/gDW/h)": [""]})
    df_fluxos = pd.DataFrame(dados_monitoramento)

    # Juntar cabeçalho + dados
    df_monitoramento_total = pd.concat(
        [df_monitoramento_total, df_header, df_fluxos],
        ignore_index=True
    )

    # ----- Top 20 fluxos -----
    fluxos_abs = sol.fluxes.abs().sort_values(ascending=False)
    top20_ids = fluxos_abs.head(20).index
    top20_fluxos = sol.fluxes[top20_ids]

    df_header_top20 = pd.DataFrame({"Reação": [header_text], "Fluxo (mmol/gDW/h)": [""]})
    df_top20 = pd.DataFrame({
        "Reação": top20_fluxos.index,
        "Fluxo (mmol/gDW/h)": top20_fluxos.values
    })

    # Juntar cabeçalho + dados
    df_top20_total = pd.concat(
        [df_top20_total, df_header_top20, df_top20],
        ignore_index=True
    )


In [5]:
import pandas as pd

# DataFrame global onde vamos guardar todos os resultados
resultados_totais = pd.DataFrame(columns=["Condicao", "Fracao", "Status", "Biomassa", "Fluxo_alvo"])

def registrar_resultado(condicao, fracao, status, biomassa, alvo):
    """Registra cada resultado em um DataFrame global"""
    global resultados_totais
    
    resultados_totais = pd.concat([
        resultados_totais,
        pd.DataFrame([{
            "Condicao": condicao,
            "Fracao": fracao,
            "Status": status,
            "Biomassa": biomassa,
            "Fluxo_alvo": alvo
        }])
    ], ignore_index=True)

def salvar_resultados(nome_arquivo="resumo_simulacoes.xlsx"):
    """Salva todos os resultados em um único Excel"""
    global resultados_totais
    resultados_totais.to_excel(nome_arquivo, index=False)
    print(f"Resultados consolidados salvos em '{nome_arquivo}'")


In [6]:
sol_biomassa = model.optimize()
biomassa_max = sol_biomassa.objective_value

print(sol_biomassa, biomassa_max)

<Solution 1.043 at 0x7f5f6bd15b70> 1.0426374979773858


In [7]:
# Glicose, Aeróbica (forçando biomassa)
# Nome da condição (ajuste conforme o bloco que está rodando)
condicao = "1_aero_biomassa_et"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        #otimizar e guardar os fluxos 
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "BIOMASS_")
        sol_biomax = model.optimize()  # biomassa máxima

        biomassa_max = sol_biomax.objective_value

        biomassa_min = frac * biomassa_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = biomassa_min

        model.objective = "EX_etoh_e"
        sol = model.optimize()
        print(f"=== Fração de biomassa: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print("")
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            # 🔹 Monitoramento detalhado do fluxos 
            monitorar_fluxos(sol, frac, biomassa_min, "EX_etoh_e") 
                
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_biomassa_E.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'")

=== Fração de biomassa: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 36.99999999999998

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.1 ===
Status: optimal
Biomassa: 0.18651538072780072
Fluxo alvo: 34.56049383789854

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.2 ===
Status: optimal
Biomassa: 0.3730307614556061
Fluxo alvo: 32.12098767579688

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.3 ===
Status: optimal
Biomassa: 0.5595461421834093
Fluxo alvo: 28.250212765957798

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.4 ===
Status: optimal
Biomassa: 0.7460615229112172
Fluxo alvo: 24.21446808510613

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.5 ===
Status: optimal
Biomassa: 0.9325769036390159
Fluxo alvo: 20.178723404255436

Resultados salvos em 'fluxos_1_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.6 ===
Status: optima

In [8]:
# Glicose, Anaeróbica (forçando biomassa)
condicao = "1_anaero_biomassa_et"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        #otimizar e guardar os fluxos 
        ajustar_meio("EX_glc__D_e", 18.5, 0, "BIOMASS_")
        sol_biomax = model.optimize()  # biomassa máxima

        biomassa_max = sol_biomax.objective_value

        biomassa_min = frac * biomassa_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = biomassa_min

        model.objective = "EX_etoh_e"
        sol = model.optimize()
        print(f"=== Fração de biomassa: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print("")
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)           
            
                  # 🔹 Monitoramento detalhado do fluxos 
            monitorar_fluxos(sol, frac, biomassa_min, "EX_etoh_e")  # 🔹 Chama a função
                
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_biomassa_E.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'")

=== Fração de biomassa: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 36.999999999999886

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.1 ===
Status: optimal
Biomassa: 0.05032259836838311
Fluxo alvo: 36.34181144561047

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.2 ===
Status: optimal
Biomassa: 0.10064519673676736
Fluxo alvo: 35.68362289122129

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.3 ===
Status: optimal
Biomassa: 0.15096779510515312
Fluxo alvo: 35.025434336831886

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.4 ===
Status: optimal
Biomassa: 0.20129039347353672
Fluxo alvo: 34.36724578244193

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.5 ===
Status: optimal
Biomassa: 0.25161299184192465
Fluxo alvo: 33.70905722805259

Resultados salvos em 'fluxos_1_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.6 ==

In [9]:
# Glicose, Aeróbica (forçando etanol) 

condicao = "1_aero_etanol"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "EX_etoh_e")
        sol_alvo = model.optimize()  # etanol máxima

        
        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_etoh_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            
            #print()
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()


    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_etanol.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_etanol.xlsx'")




=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 1.8651538072780325
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.6941551531839412
Fluxo alvo: 3.7000000000000046

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.5231564990898798
Fluxo alvo: 7.399999999999963

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.3521578449957903
Fluxo alvo: 11.100000000000032

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.1811591909017867
Fluxo alvo: 14.800000000000091

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.0101605368076123
Fluxo alvo: 18.499999999999915

Resultados salvos em 'fluxos_1_aero_etanol.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.8391618827135738
Fluxo alvo: 22.1999

In [10]:
# Glicose, Anaeróbica, etanol(forçando etanol) 

condicao = "1_anaero_etanol"


for frac in [round(i * 0.1, 1) for i in range(1, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 0, "EX_etoh_e")
        sol_alvo = model.optimize()  # etanol máxima

        alvo_max = sol_alvo.objective_value


        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_etoh_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
           
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_etanol.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'")


=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.5032259836838444
Fluxo alvo: 15.122733004101532

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.5032259836838336
Fluxo alvo: 15.122733004101601

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.503225983683691
Fluxo alvo: 15.122733004102212

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.5032259836837573
Fluxo alvo: 15.122733004101882

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.4804065573712993
Fluxo alvo: 18.499999999999776

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.45540649742371697
Fluxo alvo: 22.20000000000049

Resultados salvos em 'fluxos_1_anaero_etanol.xlsx'
=== Fração do alvo: 0.7 ===
Status: optimal
Biomassa: 0.430406437

In [11]:
#3. Glicose, Aeróbica, S,S-BDO (forçando biomassa)

condicao = "1_aero_biomassa_SS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__SS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)

            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__SS_e")  # 🔹 Chama a função
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_biomassa_SS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 18.499999999999936
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.18651538072788615
Fluxo alvo: 17.280246918948624
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.37303076145560127
Fluxo alvo: 15.643711340206004
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.5595461421834135
Fluxo alvo: 13.688247422680298
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.7460615229111952
Fluxo alvo: 11.732783505154684
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.9325769036390225
Fluxo alvo: 9.777319587628655
Resultados salvos em 'fluxos_1_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.1190922843

In [12]:
#4. Glicose, Anaeróbica, S,S-BDO (forçando biomassa)

condicao = "1_anaero_biomassa_SS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 0, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

         # Bloqueia as reações 
        model.reactions.get_by_id("DCTR").lower_bound = 0
        model.reactions.get_by_id("DCTR").upper_bound = 0
        model.reactions.get_by_id("DCTRa").lower_bound = 0
        model.reactions.get_by_id("DCTRa").upper_bound = 0
        
        
        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__SS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__SS_e")  # 🔹 Chama a função
            
          # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_biomassa_SS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'")

=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.5032259836838535
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.5032259836838411
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.5032259836838513
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.50322598368385
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.5032259836838496
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.5032259836838318
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.5032259836838309
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaer

In [13]:
#3. Glicose, Aeróbica, S,S-BDO (forçando bdo)

condicao = "1_aero_SS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "EX_btd__SS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value
        

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__SS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)            
            
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()
     

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_SS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 1.8651538072780491
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.6886977493298838
Fluxo alvo: 1.849999999999996

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.5122416913817922
Fluxo alvo: 3.7000000000000024

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.3357856334335778
Fluxo alvo: 5.54999999999999

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.1593295754854358
Fluxo alvo: 7.399999999999938

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.9828735175369865
Fluxo alvo: 9.250000000000151

Resultados salvos em 'fluxos_1_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.8064174595891759
Fluxo alvo: 11.10000000

In [14]:
#4. Glicose, Anaeróbica, S,S-BDO (forçando bdo)

condicao = "1_anaero_SS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 0, "EX_btd__SS_e")
        sol_alvo = model.optimize()  # bdo máxima

        model.reactions.get_by_id("DCTR").lower_bound = 0
        model.reactions.get_by_id("DCTR").upper_bound = 0
        model.reactions.get_by_id("DCTRa").lower_bound = 0
        model.reactions.get_by_id("DCTRa").upper_bound = 0
        
        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__SS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)            
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
            
         # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()
        
    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_SS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.5032259836838482
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'


/home/caaraujo/miniconda3/envs/cameo/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


=== Fração do alvo: 0.1 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.7 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.8 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_1_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.9 ===
Status: infeasible
Solução inviável.

In [15]:
# Glicose, Aeróbica, R,S-BDO (forçando biomassa)

condicao = "1_aero_biomassa_RS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__RS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__RS_e")  # 🔹 Chama a função
            
        # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_biomassa_RS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 18.500000000000064

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.18651538072780263
Fluxo alvo: 17.38369922916479

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.37303076145560476
Fluxo alvo: 16.26739845832932

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.5595461421834071
Fluxo alvo: 14.997318426555077

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.7460615229112043
Fluxo alvo: 12.9327272727272

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.9325769036390035
Fluxo alvo: 10.777272727272866

Resultados salvos em 'fluxos_1_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.1190922

In [16]:
# Glicose, Anaeróbica, R,S-BDO (forçando biomassa)

condicao = "1_anaero_biomassa_RS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 0, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__RS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        
        if sol.status == "optimal":
            #print()
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)

            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__RS_e")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()


    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_biomassa_RS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 17.709375000000268

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.05032259836838457
Fluxo alvo: 17.306277724181914

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.10064519673677173
Fluxo alvo: 16.90318044836382

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.15096779510515496
Fluxo alvo: 16.500083172544674

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.20129039347353994
Fluxo alvo: 16.09698589672614

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.25161299184192504
Fluxo alvo: 14.402878473322945

Resultados salvos em 'fluxos_1_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Bi

In [17]:
#3. Glicose, Aeróbica, R,S-BDO (forçando bdo)

condicao = "1_aero_RS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 1000, "EX_btd__RS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__RS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
        # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_aero_RS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 1.8651538072779827
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.7050699608921043
Fluxo alvo: 1.85

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.5449861145061299
Fluxo alvo: 3.699999999999981

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.3849022681201881
Fluxo alvo: 5.54999999999998

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.2248184217342108
Fluxo alvo: 7.399999999999974

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.064734575348316
Fluxo alvo: 9.249999999999966

Resultados salvos em 'fluxos_1_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.9046507289623477
Fluxo alvo: 11.099999999999966

Result

In [18]:
#3. Glicose, Anaeróbica, R,S-BDO (forçando bdo)

condicao = "1_anaero_RS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("EX_glc__D_e", 18.5, 0, "EX_btd__RS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__RS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
       # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_1_anaero_RS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.5032259836839014
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.47606854591721054
Fluxo alvo: 1.7709374999999943

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.44891110815059204
Fluxo alvo: 3.541875000000009

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.42175367038392436
Fluxo alvo: 5.312812500000411

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.3945962326174346
Fluxo alvo: 7.083749999999978

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.3674387948506924
Fluxo alvo: 8.854687500000031

Resultados salvos em 'fluxos_1_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.34028135708412716
Fluxo 

In [19]:
# ------------------------------- GLICEROL -------------------------------

In [20]:
# Glicerol, Aeróbica, forçando biomassa  

condicao = "2_aero_biomassa_SS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 1000, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__SS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)

            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__SS_e")  # 🔹 Chama a função
            
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()
        
    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_biomassa_SS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 19.707000000000075

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.21341182687900698
Fluxo alvo: 18.42972310240175

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.4268236537581367
Fluxo alvo: 17.15244620480241

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.640235480637191
Fluxo alvo: 15.593898389456541

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.8536473075162606
Fluxo alvo: 13.424709278350429

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.0670591343953746
Fluxo alvo: 11.187257731958194

Resultados salvos em 'fluxos_2_aero_biomassa_SS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.2804709

In [21]:
# Glicerol, Anaeróbica, forçando biomassa 

condicao = "2_anaero_biomassa_SS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 0, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__SS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__SS_e")  # 🔹 Chama a função
            
         # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()
        
        
    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_biomassa_SS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'")

=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 18.756136752136765

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.04747759033406189
Fluxo alvo: 18.340073286408984

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.09495518066805514
Fluxo alvo: 17.924009820683644

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.14243277100212684
Fluxo alvo: 17.507946354956896

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.18991036133614952
Fluxo alvo: 16.589900520019906

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.2373879516706282
Fluxo alvo: 14.27593120556058

Resultados salvos em 'fluxos_2_anaero_biomassa_SS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Bi

In [22]:
#1a.Glicerol, Aeróbica, etanol 
#with model:


In [23]:
#2a.Glicerol, Anaeróbica, etanol
#with model:


In [24]:
#3a.Glicerol, Aeróbica, forçando S,S-BDO 

condicao = "2_aero_SS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 1000, "EX_btd__SS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__SS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
              # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_SS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 2.134118268790621
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.9461496453320455
Fluxo alvo: 1.9706999999999952

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.758181021873448
Fluxo alvo: 3.9414000000000047

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.5702123984148422
Fluxo alvo: 5.912100000000041

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.382243774956236
Fluxo alvo: 7.882799999999945

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.1942751514976457
Fluxo alvo: 9.85349999999996

Resultados salvos em 'fluxos_2_aero_SS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.0063065280390047
Fluxo alvo: 11.8242000000

In [25]:
#4a.Glicerol, Anaeróbica, forçando BDO

condicao = "2_anaero_SS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 0, "EX_btd__SS_e")
        sol_alvo = model.optimize()  # bdo máxima

        model.reactions.get_by_id("DCTR").lower_bound = 0
        model.reactions.get_by_id("DCTR").upper_bound = 0
        model.reactions.get_by_id("DCTRa").lower_bound = 0
        model.reactions.get_by_id("DCTRa").upper_bound = 0
        
        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__SS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__SS_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_SS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.47477590334045044
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.7 ===
Status: infeasible
Solução inviável.

Resultados salvos em 'fluxos_2_anaero_SS_bdo.xlsx'
=== Fração do alvo: 0.8 ===
Status: inf

In [26]:
# Glicerol, Aeróbica, forçando biomassa  

condicao = "2_aero_biomassa_RS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 1000, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__RS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            
            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__RS_e")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_biomassa_RS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 19.707000000000136

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.2134118268790628
Fluxo alvo: 18.42972310240105

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.42682365375811043
Fluxo alvo: 17.152446204802395

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.6402354806372099
Fluxo alvo: 15.87516930720345

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.85364730751628
Fluxo alvo: 14.589588142554213

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.0670591343953255
Fluxo alvo: 12.331409090909073

Resultados salvos em 'fluxos_2_aero_biomassa_RS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.28047096

In [27]:
# Glicerol, Anaeróbica, forçando biomassa  

condicao = "2_anaero_biomassa_RS"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 0, "BIOMASS_")
        sol_alvo = model.optimize()  # bdo máxima

        model.reactions.get_by_id("DCTR").lower_bound = 0
        model.reactions.get_by_id("DCTR").upper_bound = 0
        model.reactions.get_by_id("DCTRa").lower_bound = 0
        model.reactions.get_by_id("DCTRa").upper_bound = 0
        
        
        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = alvo_min

        model.objective = "EX_btd__RS_e"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "EX_btd__RS_e")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()


    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_biomassa_RS.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 17.144281250000084

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.04747759033404679
Fluxo alvo: 16.763973238359164

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.09495518066810418
Fluxo alvo: 16.383665226718545

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.14243277100212684
Fluxo alvo: 15.466802591831085

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.18991036133620837
Fluxo alvo: 13.573554970923864

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.23738795167026044
Fluxo alvo: 11.680307350018476

Resultados salvos em 'fluxos_2_anaero_biomassa_RS.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal


In [28]:
# Glicose, Aeróbica, R,S-BDO (forçando bdo)

condicao = "2_aero_RS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 1000, "EX_btd__RS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__RS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()

            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)

            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
        # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_RS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 2.13411826879065
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.9635900330756213
Fluxo alvo: 1.9706999999999895

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.7930617973606175
Fluxo alvo: 3.94140000000004

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.6225335616455934
Fluxo alvo: 5.912100000000033

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.4520053259305366
Fluxo alvo: 7.882800000000088

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.2814770902155417
Fluxo alvo: 9.85349999999988

Resultados salvos em 'fluxos_2_aero_RS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 1.110948854500545
Fluxo alvo: 11.824200000000

In [29]:
# Glicose, Anaeróbica, R,S-BDO (forçando bdo)

condicao = "2_anaero_RS_bdo"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 0, "EX_btd__RS_e")
        sol_alvo = model.optimize()  # bdo máxima


        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_btd__RS_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()

            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_btd__RS_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)

        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)

        print()
        
        
    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_RS_bdo.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.4747759033404062
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.44848504038785225
Fluxo alvo: 1.7144281250000006

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.42219417743530924
Fluxo alvo: 3.4288562499999915

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.39580827910008753
Fluxo alvo: 5.143284374999991

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.3583257413750833
Fluxo alvo: 6.85771250000003

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.31533246607549825
Fluxo alvo: 8.572140625000017

Resultados salvos em 'fluxos_2_anaero_RS_bdo.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.27233919077590163
Fluxo

In [30]:
# Glicerol, Aeróbica (forçando biomassa)
# Nome da condição (ajuste conforme o bloco que está rodando)
condicao = "2_aero_biomassa_et"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        #otimizar e guardar os fluxos 
        ajustar_meio("glicerol", 18.5, 1000, "BIOMASS_")
        sol_biomax = model.optimize()  # biomassa máxima

        biomassa_max = sol_biomax.objective_value

        biomassa_min = frac * biomassa_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = biomassa_min

        model.objective = "EX_etoh_e"
        sol = model.optimize()
        print(f"=== Fração de biomassa: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print("")
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            # 🔹 Monitoramento detalhado do fluxos 
            monitorar_fluxos(sol, frac, biomassa_min, "EX_etoh_e") 
                
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    #Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_biomassa_E.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'")

=== Fração de biomassa: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 42.49733333333334

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.1 ===
Status: optimal
Biomassa: 0.21341182687906582
Fluxo alvo: 39.70603824171496

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.2 ===
Status: optimal
Biomassa: 0.42682365375812276
Fluxo alvo: 36.124623627830076

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.3 ===
Status: optimal
Biomassa: 0.6402354806371829
Fluxo alvo: 31.9571576639678

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.4 ===
Status: optimal
Biomassa: 0.8536473075162662
Fluxo alvo: 27.704993890069048

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.5 ===
Status: optimal
Biomassa: 1.0670591343953049
Fluxo alvo: 23.088595744681644

Resultados salvos em 'fluxos_2_aero_biomassa_E.xlsx'
=== Fração de biomassa: 0.6 ===
Status: optim

In [31]:
# Glicerol, Anaeróbica (forçando biomassa)
condicao = "2_anaero_biomassa_et"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        #otimizar e guardar os fluxos 
        ajustar_meio("glicerol", 18.5, 0, "BIOMASS_")
        sol_biomax = model.optimize()  # biomassa máxima

        biomassa_max = sol_biomax.objective_value

        biomassa_min = frac * biomassa_max
        model.reactions.get_by_id("BIOMASS_").lower_bound = biomassa_min

        model.objective = "EX_etoh_e"
        sol = model.optimize()
        print(f"=== Fração de biomassa: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print("")
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)           
            
                  # 🔹 Monitoramento detalhado do fluxos 
            monitorar_fluxos(sol, frac, biomassa_min, "EX_etoh_e")  # 🔹 Chama a função
                
             # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_biomassa_E.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'")

=== Fração de biomassa: 0.0 ===
Status: optimal
Biomassa: 0.0
Fluxo alvo: 42.49733333333318

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.1 ===
Status: optimal
Biomassa: 0.04747759033404659
Fluxo alvo: 41.87635572961125

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.2 ===
Status: optimal
Biomassa: 0.09495518066809883
Fluxo alvo: 41.2553781258897

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.3 ===
Status: optimal
Biomassa: 0.14243277100212437
Fluxo alvo: 40.63440052216728

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.4 ===
Status: optimal
Biomassa: 0.18991036133616584
Fluxo alvo: 40.01342291844554

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.5 ===
Status: optimal
Biomassa: 0.23738795167022328
Fluxo alvo: 39.39244531472329

Resultados salvos em 'fluxos_2_anaero_biomassa_E.xlsx'
=== Fração de biomassa: 0.6 ===
S

In [32]:
# Glicerol, Aeróbica (forçando etanol) 

condicao = "2_aero_etanol"

for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 1000, "EX_etoh_e")
        sol_alvo = model.optimize()  # etanol máxima

        
        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_etoh_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            
            #print()
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
            
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()


    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_aero_etanol.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_aero_etanol.xlsx'")




=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 2.134118268790644
Fluxo alvo: 0.0

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 1.9377132200720457
Fluxo alvo: 4.249733333333383

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 1.7413081713532084
Fluxo alvo: 8.499466666666718

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 1.544903122634922
Fluxo alvo: 12.749200000000137

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 1.3484980739162804
Fluxo alvo: 16.998933333333266

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 1.1520930251977524
Fluxo alvo: 21.248666666666665

Resultados salvos em 'fluxos_2_aero_etanol.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.9556879764791714
Fluxo alvo: 25.4984

R

In [33]:
# Glicerol, Anaeróbica, etanol(forçando etanol) 

condicao = "2_anaero_etanol"


for frac in [round(i * 0.1, 1) for i in range(0, 11)]:
    with model:
        ajustar_meio("glicerol", 18.5, 0, "EX_etoh_e")
        sol_alvo = model.optimize()  # etanol máxima

        alvo_max = sol_alvo.objective_value

        alvo_min = frac * alvo_max
        model.reactions.get_by_id("EX_etoh_e").lower_bound = alvo_min

        model.objective = "BIOMASS_"
        sol = model.optimize()
        print(f"=== Fração do alvo: {frac} ===")
        print("Status:", sol.status)
        if sol.status == "optimal":
            #print()
            biomassa = sol.fluxes["BIOMASS_"]
            alvo_fluxo = sol.fluxes["EX_etoh_e"]
            
            print("Biomassa:", biomassa)
            print("Fluxo alvo:", alvo_fluxo)
            
            monitorar_fluxos(sol, frac, alvo_min, "BIOMASS_")  # 🔹 Chama a função
           
            # 🔹 Registrar no DataFrame global
            registrar_resultado(condicao, frac, sol.status, biomassa, alvo_fluxo)
            
        else:
            print("Solução inviável.")
            registrar_resultado(condicao, frac, sol.status, None, None)
        print()

    # Salvar no Excel
    with pd.ExcelWriter("fluxos_2_anaero_etanol.xlsx") as writer:
        df_monitoramento_total.to_excel(writer, sheet_name="Monitoramento", index=False)
        df_top20_total.to_excel(writer, sheet_name="Top20", index=False)

    print("Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'")


=== Fração do alvo: 0.0 ===
Status: optimal
Biomassa: 0.47477590334044445
Fluxo alvo: 25.770668131638185

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.1 ===
Status: optimal
Biomassa: 0.474775903340466
Fluxo alvo: 25.770668131637983

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.2 ===
Status: optimal
Biomassa: 0.4747759033404533
Fluxo alvo: 25.77066813163858

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.3 ===
Status: optimal
Biomassa: 0.47477590334047814
Fluxo alvo: 25.770668131637493

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.4 ===
Status: optimal
Biomassa: 0.47477590334045217
Fluxo alvo: 25.770668131637915

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.5 ===
Status: optimal
Biomassa: 0.4747759033404651
Fluxo alvo: 25.77066813163798

Resultados salvos em 'fluxos_2_anaero_etanol.xlsx'
=== Fração do alvo: 0.6 ===
Status: optimal
Biomassa: 0.47477590

In [34]:
salvar_resultados("resumo_FBA.xlsx")

Resultados consolidados salvos em 'resumo_FBA.xlsx'
